# 第六章 数据加载、存储与文件格式

In [2]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np

# 6.1 读写分隔符文本文件

`df.read_csv()` 读入csv文件，默认分隔符为逗号

`df.read_table()` 加载带分隔符的数据，默认分割符为字表符（`\t`）, 读入逗号分隔符文件需指定分隔符。

In [40]:
df=pd.read_table('data/ch06/ex1.csv')
df

,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


In [41]:
df=pd.read_table('data/ch06/ex1.csv',sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 读入无表头文件

+ 默认第一行做表头 
- 表头设置为空：`header=None`
+ 指定表头：`names=[]`
- 指定索引列：`index_col='str'`

In [49]:
pd.read_csv('data/ch06/ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [48]:
pd.read_csv('data/ch06/ex2.csv',names=['a','v','c','d','message'],index_col='v')

,a,c,d,message
v,,,,
2,1,3,4,hello
6,5,7,8,world
10,9,11,12,foo


#### 读入文件并设置多重索引
- `.index_col=['key1','key2']`

In [50]:
pd.read_csv('data/ch06/csv_mindex.csv')

,key1,key2,value1,value2
0,one,a,1,2
1,one,b,3,4
2,one,c,5,6
3,one,d,7,8
4,two,a,9,10
5,two,b,11,12
6,two,c,13,14
7,two,d,15,16


In [51]:
pd.read_csv('data/ch06/csv_mindex.csv',index_col=['key1','key2'])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [2]:
list(open('data/ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

#### 非固定分隔符
正则表达式
- `sep='\s+'`

In [3]:
pd.read_csv('data/ch06/ex3.txt')

,A B C
0,aaa -0.264438 -1.026059 -0.619500
1,bbb 0.927272 0.302904 -0.032399
2,ccc -0.264273 -0.386314 -0.217601
3,ddd -0.871858 -0.348382 1.100491


In [4]:
pd.read_csv('data/ch06/ex3.txt',sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


#### 导入时跳行
- `skiprows=[0,2,4]`

#### 导入时缺失值表示
值为 `str` 的位置替换为 `NaN`
- `na_values=['str']`

#### 不同的行不同的缺失值表示
- `dict={'message':['foo','NA'],’something':['two']}`

In [11]:
pd.read_csv('data/ch06/ex5.csv')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [12]:
pd.read_csv('data/ch06/ex5.csv',na_values=['world'])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,foo


In [14]:
dict={'message':['foo','NA'],'something':['two']}
pd.read_csv('data/ch06/ex5.csv',na_values=dict)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 6.1.1 逐块读取文本文件
- 读取前几行
 - `nrows=5`
- 逐块读入(行数)
 - `chunksize=1000` <br>每次读1000的迭代器

In [16]:
pd.read_csv('data/ch06/ex6.csv',nrows=3)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G


In [30]:
chunker=pd.read_csv('data/ch06/ex6.csv',chunksize=1000)
chunker

In [39]:
tot=Series([])
for piece in chunker:
    tot=tot.add(piece['key'].value_counts(),fill_value=0)

`Series.add()` 实现了相同 `key` 的累加

In [34]:
tot.sort_values(ascending=False).head(5)

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
dtype: float64

### 6.1.2 数据写入到文本格式
#### `df.to_csv`
1. `df.to_csv` 默认分隔符为逗号

2. 指定分隔符： `sep='|'`

3. 忽略索引：`index=False, header=False`

4. 写入部分列并指定顺序：`columns=['a','d','c']`
5.  空字符串
 - 默认写为空
 - 写入指定字符：`na_rep='Null'`
 
注：`pd.read_csv()` 返回一个`df`

In [58]:
data=pd.read_csv('data/ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [61]:
data.to_csv('data/ch06/out.csv',sep='|',na_rep='haha',index=False,header=False,columns=['a','d','c'])

#### `Series` 与 `csv` 交互
1. `Series.to_csv()`
2. `Series.from_csv()`

`parse_dates`：默认为 `True`，将日期数据解析为日期，`False` 时日期数据解析为普通文本

### 6.1.3 手动处理分隔符数据

In [141]:
import csv
f=open('data/ch06/ex7.csv')
lines=list(csv.reader(f))
header,values=lines[0],lines[1:]
data_dict={h:v for h,v in zip(header,zip(*values)) }
DataFrame(data_dict)

,a,b,c
0,1,2,3
1,1,2,3


#### `zip` 函数
`zip(*[[list1],[list2]])`

等价于先解开再 `zip`:<br>
`zip([list1],[list2])`

In [117]:
for item in zip(['1', '2', '3'],['1', '2', '3', '4'] ):
    print(item)

('1', '1')
('2', '2')
('3', '3')


In [116]:
for item in zip(*[['1', '2', '3'],['1', '2', '3', '4']] ):
    print(item)

('1', '1')
('2', '2')
('3', '3')


#### `csv.Dialect`

In [136]:
class my_dialect(csv.Dialect):
    lineterminator='\n'
    delimiter=','
    quotechar='"'
    quoting=csv.QUOTE_ALL

In [139]:
reader=csv.reader(open('data/ch06/ex7.csv'),dialect=my_dialect)

In [140]:
for item in reader:
    print(item)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


## 6.2 Json数据
- `python` 标准库：`json`
 - `json.loads(obj)`
 - `json.dumps(results)`
- `pandas`方法
 - `pd.read_json()`
 - `df.to_json()`

In [6]:
df = pd.DataFrame([['a', 'b'], ['c', 'd']],
                  index=['row 1', 'row 2'],
                  columns=['col 1', 'col 2'])
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [4]:
df.to_json()

'{"col 1":{"row 1":"a","row 2":"c"},"col 2":{"row 1":"b","row 2":"d"}}'

In [5]:
df.to_json(orient='split')

'{"columns":["col 1","col 2"],"index":["row 1","row 2"],"data":[["a","b"],["c","d"]]}'

## 6.3 XML HTML数据

## 6.4 Web API 

## 6.5 二进制数据 

## 6.6 数据库